# Testing file 
### where we evaluate Zhang's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.data import Dataset


from util.load_data import load_data
from util.evaluation import *
from models.zhang.models import FairLogisticRegression
from models.zhang.learning import train_loop as zhang_train

In [2]:
batch_size = 64
epochs = 1
lr = 0.001
opt = Adam(learning_rate=lr)

In [3]:
test_loop = 1

## Load data

In [4]:
data_name = 'adult'

In [5]:
x_train, y_train, a_train = load_data(data_name, 'train')
raw_data = (x_train, y_train, a_train)

In [6]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [7]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 112), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [8]:
x_valid, y_valid, a_valid = load_data(data_name, 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [9]:
x_test, y_test, a_test = load_data(data_name, 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [10]:
header = "model_name", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### Zhang for DP

In [11]:
fairdef = 'DemPar'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, adim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y, A, Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4DP', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.537175178527832 | 0.8347809910774231 | 0.6528514588859416 | 0.3241462201591512
> Class Acc | Adv Acc
> 0.8068484042553191 | 0.32563164893617025
> DP | DEqOdds | DEqOpp
> 0.8780761063098907 | 0.8350172247737646 | 0.7082487344741821
> Confusion Matrix 
TN: 4451.0 | FP: 107.0 
FN: 1055.0 | TP: 403.0
> Confusion Matrix for A = 0 
TN: 1758.0 | FP: 0.0 
FN: 196.0 | TP: 5.0
> Confusion Matrix for A = 1 
TN: 2693.0 | FP: 107.0 
FN: 859.0 | TP: 398.0


### Zhang for Eq Odds

In [12]:
fairdef = 'EqOdds'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, adim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y, A, Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4EqOdds', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.5167679786682129 | 0.7780622243881226 | 0.6641661140583555 | 0.32418766578249336
> Class Acc | Adv Acc
> 0.8228058510638298 | 0.3272938829787234
> DP | DEqOdds | DEqOpp
> 0.7855367064476013 | 0.7250424064695835 | 0.5398069024085999
> Confusion Matrix 
TN: 4299.0 | FP: 259.0 
FN: 807.0 | TP: 651.0
> Confusion Matrix for A = 0 
TN: 1755.0 | FP: 3.0 
FN: 191.0 | TP: 10.0
> Confusion Matrix for A = 1 
TN: 2544.0 | FP: 256.0 
FN: 616.0 | TP: 641.0


### Zhang for Eq Opp

In [13]:
fairdef = 'EqOpp'

for i in range(test_loop):
    opt = Adam(learning_rate=lr)
    model = FairLogisticRegression(xdim, adim, batch_size, fairdef)
    zhang_train(model, raw_data, train_data, epochs, opt)

    Y, A, Y_hat, A_hat = evaluation(model, valid_data)
    clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

    fair_metrics = (dp, deqodds, deqopp)
    tradeoff = []
    for fair_metric in fair_metrics:
        tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

    result = ['Zhang4EqOpp', clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

    results.append(result)

> Epoch | Class Loss | Adv Loss | Class Acc | Adv Acc
> 1 | 0.5352369546890259 | 0.3454836308956146 | 0.6534316976127321 | 0.3241462201591512
> Class Acc | Adv Acc
> 0.8093417553191489 | 0.32563164893617025
> DP | DEqOdds | DEqOpp
> 0.8781635612249374 | 0.8410549741238356 | 0.7193982601165771
> Confusion Matrix 
TN: 4451.0 | FP: 107.0 
FN: 1040.0 | TP: 418.0
> Confusion Matrix for A = 0 
TN: 1757.0 | FP: 1.0 
FN: 192.0 | TP: 9.0
> Confusion Matrix for A = 1 
TN: 2694.0 | FP: 106.0 
FN: 848.0 | TP: 409.0


## Saving into DF then CSV

In [14]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,Zhang4DP,0.806848,0.878076,0.835017,0.708249,0.840957,0.820691,0.754340,1758.0,0.0,196.0,5.0,2693.0,107.0,859.0,398.0
1,Zhang4EqOdds,0.822806,0.785537,0.725042,0.539807,0.803739,0.770837,0.651919,1755.0,3.0,191.0,10.0,2544.0,256.0,616.0,641.0
2,Zhang4EqOpp,0.809342,0.878164,0.841055,0.719398,0.842349,0.824894,0.761724,1757.0,1.0,192.0,9.0,2694.0,106.0,848.0,409.0


In [17]:
result_df.to_csv(f'adult-result/zhang-{epochs}.csv')